In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8

## Initialize Python

In [1]:
!pip install nltk

In [2]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 3.8MB/s ta 0:00:011


In [4]:
!pip install gensim

    100% |████████████████████████████████| 24.2MB 792kB/s eta 0:00:01
    100% |████████████████████████████████| 40kB 22.7MB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 16.3MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 36.3MB/s ta 0:00:01
    100% |████████████████████████████████| 5.4MB 3.4MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 29.0MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 33.5MB/s ta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/f7/a6/ff/9ab5842c14e50e95a06a4675b0b4a689c9cab6064dac2b01d0
  Stored in directory: /home/jovyan/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [5]:
# Import general packages
import imp, importlib # For working with modules
import nltk # for natural language processing tools

import pandas as pd # for working with dataframes
#from pandas.core.groupby.groupby import PanelGroupBy # For debugging
import numpy as np # for working with numbers
import pickle # For working with .pkl files
from tqdm import tqdm # Shows progress over iterations, including in pandas via "progress_apply"
import sys # For terminal tricks
import _pickle as cPickle # Optimized version of pickle
import gc # For managing garbage collector
import timeit # For counting time taken for a process
import datetime # For working with dates & times

import csv
import re
import string
import collections


import numpy as np
from itertools import groupby
from sklearn.feature_extraction import text
import nltk
from nltk.corpus import stopwords
import datetime
import ast
from collections import Counter 

#do in terminal python3 -m pip install --upgrade pandas
import statistics

import string
folder_prefix = '/home/jovyan/work/'

# Import packages for cleaning, tokenizing, and stemming text
import re # For parsing text
from unicodedata import normalize # for cleaning text by converting unicode character encodings into readable format
from nltk import word_tokenize, sent_tokenize # widely used text tokenizer
from nltk.stem.porter import PorterStemmer # an approximate method of stemming words (it just cuts off the ends)
from nltk.stem.porter import PorterStemmer # approximate but effective (and common) method of normalizing words: stems words by implementing a hierarchy of linguistic rules that transform or cut off word endings
stem = PorterStemmer().stem # Makes stemming more accessible
from nltk.corpus import stopwords # for eliminating stop words
import gensim # For word embedding models
from gensim.models.phrases import Phrases # Makes word2vec more robust: Looks not just at  To look for multi-word phrases within word2vec
from gensim.test.utils import get_tmpfile 


# Import packages for multiprocessing
import os # For navigation
##numcpus = len(os.sched_getaffinity(0)) # Detect and assign number of available CPUs
#from multiprocessing import Pool # key function for multiprocessing, to increase processing speed
#pool = Pool(processes=numcpus) # Pre-load number of CPUs into pool function
import Cython # For parallelizing word2vec
#mpdo = False # Set to 'True' if using multiprocessing--faster for creating words by sentence file, but more complicated

# FOR VISUALIZATIONS
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns # To make matplotlib prettier

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# # FOR VISUALIZATIONS
# import matplotlib
# import matplotlib.pyplot as plt

# import seaborn as sns # To make matplotlib prettier
# sns.set(style='white')
# sns.despine()

# # Visualization parameters
# % pylab inline 
# % matplotlib inline
# matplotlib.style.use('fivethirtyeight')

In [7]:
# Define data and model file paths
charters_path = "../../nowdata/traincf_2015_15_250_counts.pkl" # All text data; only charter schools (regardless if open or not)
wordsent_path = "../data/wem_wordsent_data_train250_nostem.pkl"
phrasesent_path = "../data/wem_phrasesent_data_train250_nostem.pkl"
wem_path = "../data/wem_model_train250_nostem.txt"
wem_newpath = "../data/wem_model_train250_nostem_unlapped_300d.txt"

# Define file paths to existing dicts
ess_path = "../dicts/ess_dict.txt"
prog_path = "../dicts/prog_dict.txt"
rit_path = "../dicts/rit_dict.txt"

# Define file paths for new dicts
ess_valid_path = "../dicts/ess_valid.txt"
prog_valid_path = "../dicts/prog_valid.txt"
rit_valid_path = "../dicts/rit_valid.txt"
ess_polar = "../dicts/ess_polar.txt"
prog_polar = "../dicts/prog_polar.txt"
rit_polar = "../dicts/rit_polar.txt"

In [8]:
original_df = pd.read_pickle(folder_prefix + 'nowdata/charters_2015.pkl') #Yoon's doc2vecmodel uses this as well

In [9]:
original_df

,NCESSCH,URL,LAT1516,LON1516,AM,AS,BL,HI,HP,TR,...,PROG_VALID_STR,RIT_VALID_COUNT,RIT_VALID_STR,INQUIRY_RATIO,DISCIPLINE_RATIO,ESS_VALID_RATIO,PROG_VALID_RATIO,RIT_VALID_RATIO,in_acs,GEO_LEAID
0,1.001970e+10,http://www.maef.net/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.997944,51.0,-2.545622,0.007424,0.000446,0.000893,0.001005,0.002847,NaN,NaN
1,2.000010e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.796131,-161.765194,167.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,False,200001.0
2,2.001500e+10,https://www.kgbsd.org/ketchikancharter,55.347001,-131.641191,74.0,37.0,2.0,5.0,4.0,5.0,...,-3.123525,0.0,-6.000000,0.003762,0.000752,0.000000,0.000752,0.000000,False,200150.0
3,2.001500e+10,http://www.tongassschool.org/,55.347001,-131.641191,57.0,12.0,4.0,6.0,1.0,11.0,...,-3.752448,31.0,-2.960010,0.009768,0.000177,0.000212,0.000177,0.001096,False,200150.0
4,2.001800e+10,https://aquarian.asdk12.org/,61.192407,-149.916872,10.0,11.0,6.0,19.0,2.0,51.0,...,-6.000000,0.0,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,200180.0
5,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.198100,-149.876000,30.0,16.0,18.0,43.0,18.0,69.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,False,200180.0
6,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.001800e+10,http://www.winterberrycharterschool.com/,61.194450,-149.791641,15.0,8.0,5.0,14.0,1.0,26.0,...,-3.190948,34.0,-2.659441,0.009728,0.000193,0.000322,0.000644,0.002191,False,200180.0
9,2.001800e+10,http://www.asdk12.org/aboutschools/eagleacademy/,61.319213,-149.579442,2.0,3.0,8.0,6.0,0.0,18.0,...,-6.000000,0.0,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,200180.0


In [ ]:
original_df['WEBTEXT'][0]

In [10]:
original_df['WEBTEXT'].isna().sum() 

0

In [11]:
new_ser = []
for li in original_df['WEBTEXT']:
    if li == "":
        new_ser.append([])
    else:
        new_ser.append(li)

In [12]:
original_df['WEBTEXT'] = new_ser

In [13]:
master_webpages = []
for li in original_df['WEBTEXT']:
    for quadrup in li:
        master_webpages.append(quadrup[3])

In [14]:
len(master_webpages)

375942

In [ ]:
master_webpages[16]

In [15]:
cwd = os.getcwd()
fname = get_tmpfile(cwd + "/my_doc2vec_model_march_7")
model = gensim.models.Doc2Vec.load(fname)

In [16]:
def load_list(file_path):
    """Loads list into memory. Must be assigned to object."""
    
    textlist = []
    with open(file_path) as file_handler:
        line = file_handler.readline()
        while line:
            textlist.append(line)
            line = file_handler.readline()
    return textlist

In [17]:
inquiry_fin = [elem.strip('\n') for elem in load_list(folder_prefix + 'Charter-school-identities/dicts/inquiry.txt')]
disc_fin = [elem.strip('\n') for elem in load_list(folder_prefix + 'Charter-school-identities/dicts/discipline.txt')]

In [ ]:
inquiry_fin

In [ ]:
disc_fin

In [ ]:
#model.most_similar(positive=inquiry_fin, negative=disc_fin, topn=20) # IBL - FD

In [18]:
print("inquiry based:" ) # words core to IBL
print(model.most_similar('inquiry-based'))
print("\n")
print("problem-based:" ) # words core to IBL
print(model.most_similar('problem-based'))
print("\n")
print("discovery:" )
print(model.most_similar('discovery'))
print("\n")
print("experimental:" )
print(model.most_similar('experimental')) 
print("\n")
print("constructivist:" )
print(model.most_similar('constructivist'))

inquiry based:


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('scedu', 0.26926442980766296), ('completereview', 0.2654167115688324), ('bluemound', 0.2418098747730255), ('belimited', 0.24155358970165253), ('arbacada', 0.23936018347740173), ('thesacral', 0.23746922612190247), ('gruffalo', 0.23499497771263123), ('uniformsresources', 0.23383000493049622), ('calderbank', 0.2327917069196701), ('hersheys', 0.22779835760593414)]


problem-based:
[('zamrah', 0.24317580461502075), ('levkoff', 0.23956534266471863), ('notspace', 0.2378242313861847), ('eugnie', 0.2349020540714264), ('freesupplies', 0.2324921190738678), ('arcano', 0.2324746549129486), ('durrett', 0.2312554121017456), ('volunteersnbsp\\', 0.23108188807964325), ('llegadasalida', 0.2282860279083252), ('bidsvendors', 0.22390994429588318)]


discovery:
[('yochim', 0.2643854022026062), ('fiction', 0.25140273571014404), ('inicialmente', 0.2427150309085846), ('roadchico', 0.2384839653968811), ('wordsphrases', 0.23633471131324768), ('colt', 0.23154020309448242), ('mistress', 0.23092767596244812), ('k

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [19]:
print("discipline:" ) # words core to IBL
print(model.most_similar('discipline'))
print("\n")
print("punish:" ) # words core to IBL
print(model.most_similar('punish'))
print("\n")
print("excel:" )
print(model.most_similar('excel'))
print("\n")
print("crime:" )
print(model.most_similar('crime')) 
print("\n")
print("zero-tolerance:" )
print(model.most_similar('zero-tolerance'))

discipline:
[('herchele', 0.2700624465942383), ('netos', 0.2555854320526123), ('duncklee', 0.2524210512638092), ('visually-oriented', 0.2508392930030823), ('linkup', 0.24424222111701965), ('davlin', 0.24021190404891968), ('nelsen', 0.23797659575939178), ('promulg', 0.23666051030158997), ('connect-extend-challenge', 0.23406073451042175), ('ginoza', 0.2336428165435791)]


punish:
[('ecpvnonsgml', 0.26033955812454224), ('toexperience', 0.2544165551662445), ('braces', 0.2541964650154114), ('linacre', 0.23615995049476624), ('vietnam', 0.23525646328926086), ('wessman', 0.23252755403518677), ('franzese', 0.2302248477935791), ('gallaudet', 0.22919924557209015), ('toothbrush', 0.22832421958446503), ('ysk', 0.2260236144065857)]


excel:
[('oraa', 0.2574816048145294), ('zayya', 0.24878165125846863), ('upby', 0.2388148307800293), ('montessori-rocks', 0.23781341314315796), ('nas-network', 0.23155616223812103), ('flierliliuniform', 0.23110729455947876), ('aysc', 0.226187601685524), ('reinvigorate', 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_sim

In [ ]:
print(model.docvecs)

In [ ]:
model.wv.vocab #words in the model

In [ ]:
model.wv.vocab.keys()

In [ ]:
model.wv.index2word #word-tokens will be in most-frequent to least-frequent order

In [20]:
counts = Counter({word: vocab.count for (word, vocab) in model.wv.vocab.items()})

In [21]:
counts

Counter({'evening': 7309,
         'academy': 400975,
         "alabama's": 9,
         'first': 143612,
         'tuition-free': 11016,
         'public': 317330,
         'charter': 697369,
         'school': 2869771,
         'serves': 11583,
         'high': 390839,
         'students': 852327,
         'grades': 99943,
         '': 5298760,
         'mobile': 9913,
         'baldwin': 698,
         'counties': 2468,
         'accel': 211,
         'provides': 34572,
         'challenging': 11921,
         'college-preparatory': 2154,
         'curriculum': 250230,
         'individualized': 49093,
         'instruction': 128718,
         'small': 63483,
         'class': 239138,
         'sizes': 41672,
         'engaging': 9613,
         'use': 114240,
         'technology': 115408,
         'safe': 26206,
         'supportive': 6396,
         'environment': 48563,
         'ensure': 31535,
         'graduate': 27405,
         'college': 339779,
         'career': 69602,
        

In [22]:
print("inquiry based:" ) # words core to IBL
print(counts['inquiry-based'])
print("\n")
print("problem-based:" ) # words core to IBL
print(counts['problem-based'])
print("\n")
print("discovery:" )
print(counts['discovery'])
print("\n")
print("experimental:" )
print(counts['experimental']) 
print("\n")
print("constructivist:" )
print(counts['constructivist'])

inquiry based:
1116


problem-based:
230


discovery:
7537


experimental:
481


constructivist:
296


In [23]:
print("discipline:" ) # words core to IBL
print(counts['discipline'])
print("\n")
print("punish:" ) # words core to IBL
print(counts['punish'])
print("\n")
print("excel:" )
print(counts['excel'])
print("\n")
print("crime:" )
print(counts['crime']) 
print("\n")
print("zero-tolerance:" )
print(counts['zero-tolerance'])

discipline:
12766


punish:
102


excel:
12194


crime:
2079


zero-tolerance:
43


In [57]:
# cleaning part
# Create list of punctuation
import string # for one method of eliminating punctuation
punctuations = list(string.punctuation) # assign list of common punctuation symbols
punctuations+=['*','•','©','–','–','``','’','“','”','...','»',"''",'..._...','--','×','|_','_','§','…','⎫'] # Add a few more punctuations also common in web text
punctuations.remove('-') ; punctuations.remove("'")
#punctuations
punctstr = string.punctuation

In [ ]:
        # Filter out emails and URLs:
        if ("@" in word or word.startswith(('http', 'https', 'www', '//', '\\', 'x_', 'x/', 'srcimage')) or word.endswith(('.com', '.net', '.gov', '.org', '.jpg', '.pdf', 'png', 'jpeg', 'php'))):
            continue

In [48]:
stop_word_list = list(set(stopwords.words('english')))

#stop_word_list consists of dates and some repetitive words
for i in range(1,13):
    stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    stop_word_list.append(str(i))
stop_word_list.append('00') 
stop_word_list.append('el')
stop_word_list.append('en')
stop_word_list.append('la')
stop_word_list.append('los')
stop_word_list.append('para')
stop_word_list.append('las')
stop_word_list.append('san')
stop_word_list.append('mr')
stop_word_list.append('mrs')
stop_word_list.append('sa')
# stop_word_list.append('angeles')
# stop_word_list.append('diego')
# stop_word_list.append('california')
# stop_word_list.append('york')
stop_word_list.append('fax')
stop_word_list.append('email')
stop_word_list.append('phone')
# stop_word_list.append('harlem')
# stop_word_list.append('bronx')
# stop_word_list.append('wi')
stop_word_list.append('am')
stop_word_list.append('pm')
stop_word_list.append('org')
stop_word_list.extend(['Menu', 'Contact Us', 'Facebook', 'Calendar', 'Lunch', 'Breakfast', 'FAQs', 'FAQ'])
# stop_word_list.append('city')
# stop_word_list.append('austin')
# stop_word_list.append('antonio')
# stop_word_list.append('texas')
#adding states abbreviations for stopwords
# states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
#           "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
#           "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
#           "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
#           "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV"]
# for state in states:
#           stop_word_list.append(state)
# for state in [state.lower() for state in states]:
#           stop_word_list.append(state)

#we have text.English stopwords and nltk's english stop words
stop_words_extra = text.ENGLISH_STOP_WORDS.union(stop_word_list)

In [49]:
unicode_list  = []
for i in range(1000,3000):
    unicode_list.append(chr(i))
    
def clean_string(messy_string, rem_words, unicode_list, givepunct): 
    s = re.sub(r"\\x.*|\\u.*|\\b.*|-|\u2605$", "", messy_string.replace(u"\xa0", u" ").replace(u"\\t", u" ").strip(" "))
    s = re.sub(r'[^\x00-\x7f]',r'', s) #got rid of hex character like \xa0\ adn a\x80
    
    s = re.sub(r'^https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE) #gets rid of urls in forms https://
    #it's hard to get rid of urls when they aren't separated by space
    s = re.sub(r'|'.join(map(re.escape, unicode_list)), '', s) #should remove unicode characters
    li_text = []
    for word in re.split('\n|\t|\*|\r|\s', s): #split the string into a list of substrings split on the regex characters
        #gets rid of the rem_words, numbers, and emails
        
        # Filter out emails and URLs:
        if ("@" in word or word.startswith(('http', 'https', 'www', '//', '\\', 'x_', 'x/', 'srcimage')) or word.endswith(('.com', '.net', '.gov', '.org', '.jpg', '.pdf', 'png', 'jpeg', 'php'))):
            continue #skip this word, ignore it
            
        if ((word not in (rem_words)) and (not word.isdigit()) and ("@" not in word)) :
            li_text.append(re.sub(r'['+givepunct+']', r'', word))
    s = ' '.join(li_text) #join strings together
    s = ''.join(i for i in s if not i.isdigit()) #2nd check to remove numbers
    s = re.sub(' +', ' ',s) #remove duplicate whitespace
    s = s.strip() #remove whitespace at beginning and end of string
    return s

In [50]:
messy_string = original_df['WEBTEXT'][0][3][3]
messy_string

'e\n*\nQuestions or Comments\n*\nThis iframe contains the logic required to handle AJAX powered Gravity Forms.\nOur Location\nGet Directions\nFor:\nDriving\nWalking\nBicycling\nshow options\nhide options\n Avoid Tolls \n Avoid Highways\r                        \nFrom:\nTo:\nFetching directions......\nReset directions\nPrint directions\nEmail Us\nCarolyn Akers\nChief Executive Officer\nMelissa Dean\nDirector, K-8 Initiative\nJanelle Finley\nBranding & Development Coordinator\nEva Holloway\nOffice Manager\nJabaria Jenkins\nCareer Academies Coordinator\nJeremiah Newell\nChief Operations Officer\nMichelle Parson\nDirector, Finance and Administration\nJohn Parker\nFinance Coordinator\nJanice Rivers\nExecutive Assistant to the CEO\nChandra Scott\nDirector, Strategic Initiatives\nLee Williams\nInternship Coordinator\nHilder P. Wilson\nDirector, High School Initiatives\nEmployment Opportunities\nCurrently, there\xa0are no openings at MAEF. Thanks for your interest.\n\xa0It is the policy of the

In [58]:
clean_string(messy_string, stop_words_extra, unicode_list, punctstr)

'e Questions Comments This iframe contains logic required handle AJAX powered Gravity Forms Our Location Get Directions For Driving Walking Bicycling options hide options Avoid Tolls Avoid Highways From To Fetching directions Reset directions Print directions Email Us Carolyn Akers Chief Executive Officer Melissa Dean Director K Initiative Janelle Finley Branding Development Coordinator Eva Holloway Office Manager Jabaria Jenkins Career Academies Coordinator Jeremiah Newell Chief Operations Officer Michelle Parson Director Finance Administration John Parker Finance Coordinator Janice Rivers Executive Assistant CEO Chandra Scott Director Strategic Initiatives Lee Williams Internship Coordinator Hilder P Wilson Director High School Initiatives Employment Opportunities Currently openings MAEF Thanks interest It policy Mobile Area Education Foundation discriminate basis race color sex national origin disability age employment practices All persons hired required verify identity employment 

In [ ]:
inq_based_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        cleaned_string = clean_string(quadrup[3], stop_words_extra, unicode_list, punctstr)
        count += len(re.findall('inquiry-based', cleaned_string))
    inq_based_ser.append(count)



In [ ]:
s= "this/is/string.fds904qwnvdkf -- inquiry based"
re.findall('inquiry based', s)

In [25]:
max(inq_based_ser)

16

In [26]:
prob_based_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('problem-based', quadrup[3]))
    prob_based_ser.append(count)


In [27]:
discovery_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('discovery', quadrup[3]))
    discovery_ser.append(count)


In [28]:
exp_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('experimental', quadrup[3]))
    exp_ser.append(count)


In [29]:
const_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('contructivist', quadrup[3]))
    const_ser.append(count)


In [30]:
discipline_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('discipline', quadrup[3]))
    discipline_ser.append(count)


In [31]:
punish_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('punish', quadrup[3]))
    punish_ser.append(count)


In [32]:
excel_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('excel', quadrup[3]))
    excel_ser.append(count)


In [33]:
crime_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('crime', quadrup[3]))
    crime_ser.append(count)


In [34]:
zero_ser = []
for li in original_df['WEBTEXT']:
    count = 0
    for quadrup in li:
        count += len(re.findall('zero-tolerance', quadrup[3]))
    zero_ser.append(count)


In [35]:
original_df['inq-based'] = inq_based_ser
original_df['problem-based'] = prob_based_ser
original_df['discovery'] = discovery_ser
original_df['experimental'] = exp_ser
original_df['construct']  = const_ser
original_df['discipline'] = discipline_ser
original_df['punish'] = punish_ser
original_df['excel'] = excel_ser
original_df['crime'] = crime_ser
original_df['zero-tol'] = zero_ser

In [36]:
original_df

,NCESSCH,URL,LAT1516,LON1516,AM,AS,BL,HI,HP,TR,...,inq-based,problem-based,discovery,experimental,construct,discipline,punish,excel,crime,zero-tol
0,1.001970e+10,http://www.maef.net/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,2,0,0
1,2.000010e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.796131,-161.765194,167.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2.001500e+10,https://www.kgbsd.org/ketchikancharter,55.347001,-131.641191,74.0,37.0,2.0,5.0,4.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,2.001500e+10,http://www.tongassschool.org/,55.347001,-131.641191,57.0,12.0,4.0,6.0,1.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,2.001800e+10,https://aquarian.asdk12.org/,61.192407,-149.916872,10.0,11.0,6.0,19.0,2.0,51.0,...,0,0,0,0,0,0,0,0,0,0
5,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.198100,-149.876000,30.0,16.0,18.0,43.0,18.0,69.0,...,0,0,0,0,0,0,0,0,0,0
6,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
7,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8,2.001800e+10,http://www.winterberrycharterschool.com/,61.194450,-149.791641,15.0,8.0,5.0,14.0,1.0,26.0,...,0,0,1,0,0,3,0,1,0,0
9,2.001800e+10,http://www.asdk12.org/aboutschools/eagleacademy/,61.319213,-149.579442,2.0,3.0,8.0,6.0,0.0,18.0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
max(inq_based_ser)

16

In [38]:
li_cols= [inq_based_ser, prob_based_ser, discovery_ser, exp_ser, const_ser, discipline_ser, punish_ser, excel_ser, crime_ser, zero_ser]
st_li = ["inq-based", 'prob-based', 'discovery', 'experimental', 'constructivist', 'discipline', 'punish', 'excel', 'crime', "zero-tolerance"]
i = 0
for li in li_cols:
    print(st_li[i] + " : " + str(max(li)))
    i+=1

inq-based : 16
prob-based : 6
discovery : 205
experimental : 15
constructivist : 2
discipline : 186
punish : 26
excel : 361
crime : 158
zero-tolerance : 2


## Define helper functions

In [ ]:
def quickpickle_load(picklepath):
    '''Very time-efficient way to load pickle-formatted objects into Python.
    Uses C-based pickle (cPickle) and gc workarounds to facilitate speed. 
    Input: Filepath to pickled (*.pkl) object.
    Output: Python object (probably a list of sentences or something similar).'''

    with open(picklepath, 'rb') as loadfile:
        
        gc.disable() # disable garbage collector
        outputvar = cPickle.load(loadfile) # Load from picklepath into outputvar
        gc.enable() # enable garbage collector again
    
    return outputvar

In [ ]:
def write_list(file_path, textlist):
    """Writes textlist to file_path. Useful for recording output of parse_school()."""
    
    with open(file_path, 'w') as file_handler:
        
        for elem in textlist:
            file_handler.write("{}\n".format(elem))
    
    return    

In [ ]:
def load_list(file_path):
    """Loads list into memory. Must be assigned to object."""
    
    textlist = []
    with open(file_path) as file_handler:
        line = file_handler.readline()
        while line:
            textlist.append(line)
            line = file_handler.readline()
    return textlist

In [ ]:
def load_dict(custom_dict, file_path):
    """
    Loads in a dictionary from a file.
    Each word is stemmed and lower-cased in preparation for matching with word2vec.
    Adds each entry from the dict at file_path to the defined set custom_dict (the input), 
    which can also be an existing dictionary. This allows the creation of combined dictionaries!
    Multi-word dictionary elements (phrases) are transformed into a single word by
    replacing spaces with underscores, e.g. "college oriented" -> "college_oriented".
    """

    with open(file_path) as file_handler:
        line = file_handler.readline()
        while line:
            line = line.replace("\n", "") # Replace newlines at start, to avoid confusing tokenizers
            linetoks = word_tokenize(line)
            
            if len(linetoks) > 1: # For phrases (>1 word long) in dictionary
                line = [PorterStemmer().stem(word.lower()) for word in linetoks]
                line = "_".join(word for word in line)
                custom_dict.add(line.replace("_","")) # Also add element with no space between words
            else:
                line = PorterStemmer().stem(line.lower()) # For single-word dict elements, just stem it
            
            if "-" in line:
                custom_dict.add(line.replace("-","_")) # If appropriate, also add element that replaces "-" with "_"
                custom_dict.add(line.replace("-","")) # Also add element with no space between words
            
            custom_dict.add(line) # Add line after stemming dictionary entries and eliminating newlines
            line = file_handler.readline() # Look for anything else in that line, add that too
            
    return custom_dict

In [ ]:
ess_dict, prog_dict, rit_dict, ess_valid, prog_valid, rit_valid = set(), set(), set(), set(), set(), set()

ess_dict = load_dict(ess_dict, ess_path)
prog_dict = load_dict(prog_dict, prog_path)
rit_dict = load_dict(rit_dict, rit_path)

ess_valid = load_dict(ess_valid, ess_valid_path)
prog_valid = load_dict(prog_valid, prog_valid_path)
rit_valid = load_dict(rit_valid, rit_valid_path)

In [ ]:
def make_dicts(paths_list):
    '''Create ideology dictionaries: essentialism, progressivism, and ritualism.
    Input: List of paths to all files holding dictionaries.
    Output: List of dictionaries, each of which is a list of terms.'''
    
    keydicts = [] # Initialize list holding all dicts
    
    for path in paths_list:
        thisdict = set() # Initialize variable holding terms
        thisdict = load_dict(thisdict, path) # Load from file into variable
        keydicts.append(thisdict) # Add dict to list of dicts
    
    #ess_dict, prog_dict, rit_dict, all_ideol, all_dicts = set(), set(), set(), set(), set()

    #all_ideol = load_dict(all_ideol, ess_path)
    #all_ideol = load_dict(all_ideol, prog_path) # For complete ideological list, append second ideological dict
    #all_dicts = load_dict(all_ideol, rit_path) # For complete dict list, append ritual dict terms too

    #ess_dict = load_dict(ess_dict, ess_path)
    #prog_dict = load_dict(prog_dict, prog_path)
    #rit_dict = load_dict(rit_dict, rit_path)
    
    #keydicts = [ess_dict, prog_dict, rit_dict]
    
    print("Lists of key terms (dictionaries) loaded!")
    
    return(keydicts)

In [ ]:
def dict_cohere(thisdict, wem_model):
    '''Computes the average cosine similarity score of terms within one dictionary with all other terms in that same dictionary,
    effectively measuring the coherence of the dictionary.
    ...question for development: does it make sense to compare the average cosine similarity score between all terms 
    in thisdict and the average cosine similarity among the total model vocabulary? (Could that be, by definition, 0?)
    
    NOTE: For an unknown reason, calling this function deletes terms from thisdict.
    
    Inputs: List of key terms, word2vec model.
    Output: Average cosine similarity score of each word with all other words in the list of key terms.'''
    
    # Initialize average distance variables:
    word_avg_dist = 0
    word_avg_dists = 0
    dict_avg_sim = 0
    all_avg_dists = 0
    model_avg_dists = 0
    
    # Compute average cosine similarity score of each word with other dict words:
    for word in thisdict:
        word_avg_dist = (wem_model.distances(word, other_words=thisdict).sum())/len(thisdict) # Total diffs of word with all other words, take average
        word_avg_dists += word_avg_dist # Add up each average distance, incrementally
    dict_avg_sim = 1 - word_avg_dists/len(thisdict) # Find average cosine similarity score by subtracting avg. distance from 1

    # For comparison, compute average cosine similarity score of each word with ALL other words in the model vocabulary:
    #for word in thisdict:
    #    all_avg_dist = (wem_model.distances(word).sum())/len(model.vocab) # Default is to compare each word with all words
    #    all_avg_dists += all_avg_dist
    #model_avg_dist = 1 - all_avg_dists/len(model.vocab) # Find average cosine similarity score by subtracting avg. distance from 1

    #print("Average cosine similarities by word for this dictionary:       \t" + str(dict_avg_dist))
    #print("Compare to avg. cosine similarities by dict words to ALL words:\t" + str(model_avg_dist))
    
    return dict_avg_sim

In [ ]:
def dict_oppose(dict1, dict2, wem_model):
    '''Computes the average cosine similarity score of terms in dict1 with all terms in dict2,
    effectively measuring the opposition/non-coherence between the two dictionaries.
    
    NOTE: For an unknown reason, calling this function deletes terms from thisdict.
    
    Inputs: List of key terms, word2vec model.
    Output: Average cosine similarity score of each word in dict1 with all words in dict2.'''
    
    # Initialize average distance variables:
    word_avg_dist = 0
    word_avg_dists = 0
    dicts_avg_dist = 0
    
    # Compute average cosine similarity score of each term in dict1 with all terms in dict2:
    for word in dict1:
        word_avg_dist = (wem_model.distances(word, other_words=dict2).sum())/len(dict2) # Total diffs of word with all other words, take average
        word_avg_dists += word_avg_dist # Add up each average distance, incrementally
    dicts_avg_dist = word_avg_dists/len(dict1) # Find average cosine similarity score by subtracting avg. distance from 1
    
    return dicts_avg_dist

In [ ]:
def focus_dict(thisdict, coredict, maxlen, wem_model):
    '''Focus thisdict by removing least similar word vectors until reaching maxlen.
    If any words from coredict get removed, compensate for fact that they will get added back in.
    
    Input: A list of terms, core terms not to remove, desired length, and word2vec model.
    Output: The input list focused down to desired length, and still containing all the core terms.'''

    core_count = 0 # Counts number of coredict terms that were removed
    extend_count = 0 # Counts number of terms removed to offset the coming boost of core terms (that were removed and will be added back in)

    while len(thisdict) > maxlen: # Narrow thisdict down to maxlen
        badvar = model.doesnt_match(thisdict) # Find least matching term
        thisdict.remove(badvar) # Remove that least focal term, to focus dict
        if badvar in coredict: # Keep track of number of core terms removed
            core_count += 1

    while extend_count < core_count: # Remove terms until length = maxlen - number of core terms removed (to offset those core terms that will be added back in later in this script)
        badvar = model.doesnt_match(thisdict) # Find least matching term
        thisdict.remove(badvar) # Remove that least focal term, to focus dict
        extend_count += 1 # Keep track of # non-core terms added
        if badvar in coredict: # Keep track of number of core terms removed
            core_count += 1
            
    for term in coredict: # Add back in any missing core terms
        if term not in thisdict:
            thisdict.append(term)
            
    thisdict = list(set(thisdict)) # Remove any duplicates
    
    if len(thisdict) != maxlen: # Quality check
        print("WARNING: Function produced a dictionary of length " + str(len(thisdict)) + \
              ", which is not the specified maximum dict length of " + str(maxlen))
    
    return thisdict 

In [ ]:
def clean_textlist(textlist):
    """Removes extraneous spaces: at beginning and end of word2vec-format list of strings, 
    and between two parts of word at start of each line.
    
    Input: list of strings in word2vec format, e.g. ["aardvark .40 .01 .21...\n", "banana .25 .71 .06...\n", ...].
    Output: cleaned list of strings in word2vec format, which ideally should work for word2vec model generation."""
    
    unspaced_text = [textlist[0].replace("\n", "")]

    for oneline in textlist[1:]:
        first, pieces, line = "", [], []
        
        if "\\" in oneline:
            continue # Skip lines with URLs, etc.
        
        for piece in oneline.strip(" ").split(" "): # Remove spaces from beginning and end, then split by spaces
            if piece in ["", " ", "1.", "2.", "3.", "4.", "5.", "6.", "7.", "8.", "9.", "10."]:  # ["", " "] or (piece.replace(".", "").isdigit() and literal_eval(piece) >= 1):
                continue # Omit empty or erroneous elements (1.0 or greater)
            elif not piece.replace("-", "").strip("\n").replace(".", "").replace("e", "").isdigit(): # Check if digit (after removing negative signs, decimal points, scientific notation, and newlines)
                first += piece # If not a digit, it's part of the word, so concatenate it with rest of word
            else:
                pieces.append(piece.replace("\n", "")) # If a digit, add it to list of digits in vector

        line = (first + " " + " ".join([piece for piece in pieces])).strip(" ") # Reconstruct vector
        unspaced_text.append(line) # Add to new word2vec text list/ object
    
    return unspaced_text

## Refine & Validate the Dictionaries

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(wem_newpath) # Load word2vec model

In [ ]:
# dictionary of words in model (may not work for old gensim)
print(len(model.vocab))
sorted(list(model.vocab)[:200])

### Fine-tuning existing INQUIRY dicts

In [ ]:
# Define 30 core terms:
inq30 = ['discovery-based', 'student-driven_exploration', 'exploration_and_experimentation', 'laboratory-based', 
         'problem-based', 'prbl', 'learn-by-doing', 
         'project-based', 'project-centered', 
         'experiential', 'experiential_approach', 'experientially',
         'inquiry-based', 'inquiry-driven', 'student-centered_inquiry-based', 'active_inquiry', 
         'constructivist', 'constructivism', 
         'hands-on', 'hand-on', 'hands-on_learning', 'hands-on_and_minds-on', 'hands-on_minds-on', 'hands-on/minds-on', 
         'socratic', 'socratic_method', 'socratic_dialogue',
         'child-centered', 'learner-centered', 'student-centered']

In [ ]:
# Load completed dict of 500 terms:
inquiry_fin = [elem.strip('\n') for elem in load_list('../dicts/inquiry.txt')]

sorted(inquiry_fin)

In [ ]:
# Remove any terms from core dict NOT in current model (these will have to be replaced):
for word in inq30:
    if word not in list(model.vocab):
        inq30.remove(word)
        print("NOT FOUND, REMOVING:\t" + word)
    else:
        #print("FOUND, KEEPING:\t\t" + word)
        pass
    
print("\nDONE")

In [ ]:
# Remove any terms from dict NOT in current model (these will have to be replaced):
for word in inquiry_fin:
    if word not in list(model.vocab):
        inquiry_fin.remove(word)
        print("NOT FOUND, REMOVING:\t" + word)
    else:
        #print("FOUND, KEEPING:\t\t" + word)
        pass
    
print("\nDONE")

In [ ]:
inquiry_fin = list(set(inquiry_fin)) # Remove duplicates

print("Length of original dict: ", len(inq30))
print("Coherence of original dict: ", str(dict_cohere(inq30, model)))

print("Length of extended dict: ", len(inquiry_fin))
print("Coherence of extended dict: ", str(dict_cohere(inquiry_fin, model)))

In [ ]:
# Find and display terms (not cosine sim scores) similar to thisdict:
addmore = []
addmore = [tup[0] for tup in model.most_similar(inquiry_fin, topn=200)]
addmore

In [ ]:
# Find the dict term that fits the others the least (maybe to delete it, to maximize dict coherence):
badterm = model.doesnt_match(inquiry_fin)
badterm

In [ ]:
#inq_write.remove(badterm)

print(len(inquiry_fin))
print(dict_cohere(inquiry_fin, model))

In [ ]:
inq470 = inquiry_fin

for term in inq30:
    if term in inq470:
        inq470.remove(term)
        
inq_write = sorted(inq30) + sorted(inq470)

inq_write

In [ ]:
#write_list("../dicts/inquiry.txt", inq_write)

### Looking for STEM cluster

In [ ]:
stem = ['stem', 
        'technology_engineering', 
                   'art+design', 
                   'steam', 
                   'cstem', 
                   'stemr', 
                   's.t.e.a.m.s', 
                   'steams_expo', 
                   'creators_and_innovators', 
                   'mathematical_experiments', 
                   'multiliteracy', 
                   'biology/environmental', 
                   'kits/manipulators', 
                   'steam_is_an_acronym', 
                   'ap_enviromental', 
                   'biology_earth/space']

print(dict_cohere(stem, model))
print(dict_oppose(stem, inq30, model))
print(dict_oppose(stem, disc30, model))
model.most_similar(stem, topn=50)

### Fine-tuning existing DISCIPLINE dict

In [ ]:
# Define 30 core terms:
disc30 = ['discipline', 'disciplinary', 
          'no-excuses', 'zero-tolerance', 'zero_tolerance', 'zero-tolerance_policy', 
          'expel', 'expellable', 'expulsion', 'suspension', 'suspended', 
          'misbehavior', 'disobedience', 'absenteeism', 'inappropriate', 
          'perpetrator', 'crime', 'criminal', 'illegal', 'drugs', 'drug-related', 
          'violation', 'violate', 'penalty', 'punish', 'penalize', 
          'authority', 'deterrence', 'deter', 
          'behavioral_expectations']

In [ ]:
# Load completed dict of 500 terms:
disc_fin = [elem.strip('\n') for elem in load_list('../dicts/discipline.txt')]
sorted(disc_fin)

In [ ]:
# Remove any terms from dict NOT in this model's vocab:
for word in disc_fin:
    if word not in list(model.vocab):
        disc_fin.remove(word)
        print("NOT FOUND, REMOVING:\t" + word)
    else:
        #print("FOUND, KEEPING:\t\t" + word)
        pass

print("\nDONE")

In [ ]:
disc_fin = list(set(disc_fin)) # Remove duplicates

print("Length of original dict: ", len(disc30))
print("Coherence of original dict: ", str(dict_cohere(disc30, model)))

print("Length of extended dict: ", len(disc_fin))
print("Coherence of extended dict: ", str(dict_cohere(disc_fin, model)))

In [ ]:
# Find the dict term that fits the others the least (maybe to delete it, to maximize dict coherence):
badterm = model.doesnt_match(disc_fin)
badterm

In [ ]:
#disc_write.remove(badterm)
print(len(disc_fin))
print(dict_cohere(disc_fin, model))

In [ ]:
# Find and display terms (not cosine sim scores) similar to thisdict:
addmore = []
addmore = [tup[0] for tup in model.most_similar(disc_fin, topn=200)]
addmore

In [ ]:
disc470 = disc_fin

for term in disc30:
    if term in disc470:
        disc470.remove(term)
        
disc_write = sorted(disc30) + sorted(disc470)

disc_write

In [ ]:
#write_list("../dicts/discipline.txt", disc_write)

In [ ]:
discnew = ['discipline', 'disciplinary', 
          'no-excuses', 'zero-tolerance', 'zero_tolerance', 'zero-tolerance_policy', 
          'expel', 'expellable', 'expulsion', 'suspension', 'suspended', 
          'misbehavior', 'disobedience', 'absenteeism', 'inappropriate', 
          'perpetrator', 'crime', 'criminal', 'illegal', 'drugs', 'drug-related', 
          'violation', 'violate', 'penalty', 'punish', 'penalize', 
          'authority', 'deterrence', 'deter', 
          'behavioral_expectations']

In [ ]:
discnew = ['deter',
           'zero_tolerance',
           'expel',
           'suspended',
           'expulsion',
           'no-excuses',
           'violate',
           'misbehavior',
           'zero-tolerance',
           'deterrence',
           'penalty',
           'perpetrator',
           'penalize',
           'disobedience',
           'violation',
           'punish',
           'zero-tolerance_policy',
           'expellable',
           'drug-related',
           'suspension', 
          'warranting', 
          'transgression', 
          'condones', 
          'false_accusation', 
          'aversive_techniques', 
          'disciplined_appropriately', 
          'reasonably_conclude', 
          'perpetrate', 
          'revocation_of_privileges', 
          'tobacco-related']

print(str(dict_cohere(discnew, model)))
print(len(discnew))
discnew = list(set(discnew))
model.most_similar(discnew, topn=50)

### Compare dicts in vector space

In [ ]:
# Define 30 core terms:
inq30 = ['discovery-based', 'student-driven_exploration', 'exploration_and_experimentation', 'laboratory-based', 
         'problem-based', 'prbl', 'learn-by-doing', 
         'project-based', 'project-centered', 
         'experiential', 'experiential_approach', 'experientially',
         'inquiry-based', 'inquiry-driven', 'student-centered_inquiry-based', 'active_inquiry', 
         'constructivist', 'constructivism', 
         'hands-on', 'hand-on', 'hands-on_learning', 'hands-on_and_minds-on', 'hands-on_minds-on', 'hands-on/minds-on', 
         'socratic', 'socratic_method', 'socratic_dialogue',
         'child-centered', 'learner-centered', 'student-centered']

disc30 = ['discipline', 'disciplinary', 
          'no-excuses', 'zero-tolerance', 'zero_tolerance', 'zero-tolerance_policy', 
          'expel', 'expellable', 'expulsion', 'suspension', 'suspended', 
          'misbehavior', 'disobedience', 'absenteeism', 'inappropriate', 
          'perpetrator', 'crime', 'criminal', 'illegal', 'drugs', 'drug-related', 
          'violation', 'violate', 'penalty', 'punish', 'penalize', 
          'authority', 'deterrence', 'deter', 
          'behavioral_expectations']

inquiry_fin = [elem.strip('\n') for elem in load_list('../dicts/inquiry.txt')]
disc_fin = [elem.strip('\n') for elem in load_list('../dicts/discipline.txt')]

In [ ]:
# Use random adds to create a few dicts of different lengths from max length of 500:

print("TESTING DIFFERENT LENGTHS USING RANDOM ADDITIONS:\n")

inqlist, disclist = [], []; inqlist = inq30; disclist = disc30 # initialize placeholder for each growing dict
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq30, disc30, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq30, model)) + ", " + str(dict_cohere(disc30, model)))
print()

# 40 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq30, inquiry_fin)], size=10, replace=False))
#of the  words in inq_fin, if word not already in inq30 -> add it to the inquiry list
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc30, disc_fin)], size=10, replace=False))
#of the  words in disc_fin, if word not already in disc30 -> add it to the disc list

inq40, disc40 = [], []; inq40 = inqlist; disc40 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq40, disc40, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq40, model)) + ", " + str(dict_cohere(disc40, model)))
print()

# 50 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq40, inquiry_fin)], size=10, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc40, disc_fin)], size=10, replace=False))
inq50 = inqlist; disc50 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq50, disc50, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq50, model)) + ", " + str(dict_cohere(disc50, model)))
print()

# 75 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq50, inquiry_fin)], size=25, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc50, disc_fin)], size=25, replace=False))
inq75 = inqlist; disc75 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq75, disc75, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq75, model)) + ", " + str(dict_cohere(disc75, model)))
print()

# 100 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq75, disc_fin)], size=25, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc75, disc_fin)], size=25, replace=False))
inq100 = inqlist; disc100 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq100, disc100, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq100, model)) + ", " + str(dict_cohere(disc100, model)))
print()

# 200 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq100, disc_fin)], size=100, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc100, disc_fin)], size=100, replace=False))
inq200 = inqlist; disc200 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq200, disc200, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq200, model)) + ", " + str(dict_cohere(disc200, model)))
print()

# 300 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq200, disc_fin)], size=100, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc200, disc_fin)], size=100, replace=False))
inq300 = inqlist; disc300 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq300, disc300, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq300, model)) + ", " + str(dict_cohere(disc300, model)))
print()

# 400 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq300, disc_fin)], size=100, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc300, disc_fin)], size=100, replace=False))
inq400 = inqlist; disc400 = disclist
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))
print(str(len(inqlist)) + "-term dicts have this much opposition: ", str(dict_oppose(inq400, disc400, model)))
print(str(len(inqlist)) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inq400, model)) + ", " + str(dict_cohere(disc400, model)))
print()

print("500-term dicts have this much opposition: ", str(dict_oppose(inquiry_fin, disc_fin, model)))
print("500-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inquiry_fin, model)) + ", " + str(dict_cohere(disc_fin, model)))

In [ ]:
# Use removal of least similar word vectors to create a few dicts of different lengths from max length of 500:
#inqlist = inq30; disclist = disc30 # initialize placeholder for each growing dict

lens_list = [500, 400, 300, 200, 100, 75, 60, 50, 40, 30] # Define desired dict lengths

# Load full 500-term dicts:
inquiry_fin = [elem.strip('\n') for elem in load_list('../dicts/inquiry.txt')]
disc_fin = [elem.strip('\n') for elem in load_list('../dicts/discipline.txt')]

print('TESTING DICT LENGTHS BY REMOVING LEAST SIMILAR WORD VECTORS:\n')

for dictlen in lens_list: 
    inqdict, discdict = [], []
    
    inqdict = focus_dict(inquiry_fin, inq30, dictlen, model)
    discdict = focus_dict(disc_fin, disc30, dictlen, model)
    
    print(str(dictlen) + "-term dicts have this much opposition: ", str(dict_oppose(inqdict, discdict, model)))
    print(str(dictlen) + "-term dicts have this much coherence (IBL, FD): ", str(dict_cohere(inqdict, model)) + ", " + str(dict_cohere(discdict, model)))
    print()

## Surprising trends in analyzing how dictionary length affects inter-dictionary opposition and within-dictionary coherence:
### Greater lengths make for more within-dictionary coherence, but less across-dictionary opposition.
### Trimming word vectors randomly yields dictionaries that are less coherent and more opposing than does trimming those word vectors least similar to the others in the dictionary.
#### Caveat: The above patterns assume (perhaps consequentially, perhaps not) that as we trim dictionaries, we are keeping a small, core list of concepts (here, 30) intact within each dictionary. The size of this core may influence the above dynamics, as may the relative size of the total dictionary compared to the core list.

In [ ]:
model.most_similar(positive=inquiry_fin, negative=disc_fin, topn=20) # IBL - FD

In [ ]:
model.most_similar(negative=inquiry_fin, positive=disc_fin, topn=20) # FD - IBL

In [ ]:
# Let's look at two vectors that demonstrate the binary between these ideologies
print(model.most_similar('inquiry-based')) # words core to IBL
print()
print(model.most_similar('zero-tolerance')) # words core to FD

In [ ]:
# Define 30 core terms:
inq30 = ['discovery-based', 'student-driven_exploration', 'exploration_and_experimentation', 'laboratory-based', 
         'problem-based', 'prbl', 'learn-by-doing', 
         'project-based', 'project-centered', 
         'experiential', 'experiential_approach', 'experientially',
         'inquiry-based', 'inquiry-driven', 'student-centered_inquiry-based', 'active_inquiry', 
         'constructivist', 'constructivism', 
         'hands-on', 'hand-on', 'hands-on_learning', 'hands-on_and_minds-on', 'hands-on_minds-on', 'hands-on/minds-on', 
         'socratic', 'socratic_method', 'socratic_dialogue',
         'child-centered', 'learner-centered', 'student-centered']

disc30 = ['discipline', 'disciplinary', 
          'no-excuses', 'zero-tolerance', 'zero_tolerance', 'zero-tolerance_policy', 
          'expel', 'expellable', 'expulsion', 'suspension', 'suspended', 
          'misbehavior', 'disobedience', 'absenteeism', 'inappropriate', 
          'perpetrator', 'crime', 'criminal', 'illegal', 'drugs', 'drug-related', 
          'violation', 'violate', 'penalty', 'punish', 'penalize', 
          'authority', 'deterrence', 'deter', 
          'behavioral_expectations']

# Load full 500-term dicts:
inquiry_fin = [elem.strip('\n') for elem in load_list('../dicts/inquiry.txt')]
disc_fin = [elem.strip('\n') for elem in load_list('../dicts/discipline.txt')]

In [ ]:
# Let's work with the binary between IBL and FD
# first let's find the 30 words closest to each philosophy using the two 30-term dictionaries defined above
inq_words = model.most_similar("inquiry-based", topn=50)
inq_words = [word for word, similarity in inq_words]
#for word in progressivism:
#    prog_words.append(word)
print(inq_words[:20])

In [ ]:
disc_words = model.most_similar("zero-tolerance", topn=50) # now let's get the 30 most similar words for our FD dictionary
disc_words = [word for word, similarity in disc_words]
#for word in essentialism:
#    ess_words.append(word)
print(disc_words[:20])

In [ ]:
# Define 30 core terms:
inq30 = ['discovery-based', 'student-driven_exploration', 'exploration_and_experimentation', 'laboratory-based', 
         'problem-based', 'prbl', 'learn-by-doing', 
         'project-based', 'project-centered', 
         'experiential', 'experiential_approach', 'experientially',
         'inquiry-based', 'inquiry-driven', 'student-centered_inquiry-based', 'active_inquiry', 
         'constructivist', 'constructivism', 
         'hands-on', 'hand-on', 'hands-on_learning', 'hands-on_and_minds-on', 'hands-on_minds-on', 'hands-on/minds-on', 
         'socratic', 'socratic_method', 'socratic_dialogue',
         'child-centered', 'learner-centered', 'student-centered']

disc30 = ['discipline', 'disciplinary', 
          'no-excuses', 'zero-tolerance', 'zero_tolerance', 'zero-tolerance_policy', 
          'expel', 'expellable', 'expulsion', 'suspension', 'suspended', 
          'misbehavior', 'disobedience', 'absenteeism', 'inappropriate', 
          'perpetrator', 'crime', 'criminal', 'illegal', 'drugs', 'drug-related', 
          'violation', 'violate', 'penalty', 'punish', 'penalize', 
          'authority', 'deterrence', 'deter', 
          'behavioral_expectations']

core_words = inq30 + disc30
inquiry_fin = [elem.strip('\n') for elem in load_list('../dicts/inquiry.txt')]
disc_fin = [elem.strip('\n') for elem in load_list('../dicts/discipline.txt')]

# Use random adds to create a few dicts of different lengths from max length of 500:

inqlist, disclist = [], []; inqlist = inq30; disclist = disc30 # initialize placeholder for each growing dict
#print("Dict lengths (IBL, FD): " + str(len(inqlist)) + ", " + str(len(disclist)))

# 40 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq30, inquiry_fin)], size=10, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc30, disc_fin)], size=10, replace=False))
inq40, disc40 = [], []; inq40 = inqlist; disc40 = disclist

In [ ]:
# 50 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq40, inquiry_fin)], size=10, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc40, disc_fin)], size=10, replace=False))
inq50 = inqlist; disc50 = disclist

In [ ]:
# 75 terms:
inqlist.extend(np.random.choice([term for term in filter(lambda x: x not in inq50, inquiry_fin)], size=25, replace=False))
disclist.extend(np.random.choice([term for term in filter(lambda x: x not in disc50, disc_fin)], size=25, replace=False))
inq75 = inqlist; disc75 = disclist

In [ ]:
# construct a combined dictionary
id_words = disclist + inqlist
print(len(id_words))
#print(len(core_words))

## Analyzing the Model: Creating a Binary

In [ ]:
# preparing for visualizing this binary with word2vec
x = [model.similarity('inquiry-based', word) for word in id_words]
y = [model.similarity('expel', word) for word in id_words]

In [ ]:
# here's a visual of the progressivism/essentialism binary: 
# top-left half is essentialism, bottom-right half is progressivism
fig, ax = plt.subplots(figsize=(20,20))
ax.scatter(x, y, alpha=1, color='b')
for i in range(len(id_words)):
    if id_words[i] in core_words:
        ax.annotate(id_words[i], (x[i], y[i]), fontweight='bold', color="blue").set_fontsize(16)
    else:
        ax.annotate(id_words[i], (x[i], y[i])).set_fontsize(16)
ax.set_xlim(.05, .55)
ax.set_xlabel('Cosine similarity with word "inquiry-based"')
ax.set_ylim(.0, .55)
ax.set_ylabel('Cosine similarity with word "expel"')
#ax.set_title()
ax.yaxis.label.set_fontsize(24)
ax.xaxis.label.set_fontsize(24)
for item in (ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)
plt.plot([0, 1], [0, 1], linestyle='--');

### Binary of formal discipline (top-left) and inquiry-based learning (bottom-right) word vectors

In [ ]:
fig.savefig("../data/WEM-binary_IBL-vs-FD_100318.png")